In [ ]:
from utils import *
from datasets import *
from mdav import *
from train import *
from models import *
from attacks import *
from dp_data.load_dp_cifar10_dataset import *

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import dataset, DataLoader, TensorDataset, Subset
from torchvision import datasets, transforms

import random
import time
import copy
import pickle
from collections import Counter

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import scipy
import csv

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


%matplotlib inline

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning, FitFailedWarning

# Filter out ConvergenceWarning and FitFailedWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FitFailedWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
def seed_everything(seed=7):
    np.random.seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=7)

In [4]:
# Load and preprocess data

transform_train = transforms.Compose([  
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

num_classes = 10
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Randomly sample retain and forget sets
y_train = trainset.targets
forget_ratio = 0.05
idxs = np.arange(len(y_train))
random.shuffle(idxs)

m = int(len(trainset) * forget_ratio)  # Number of samples to forget
retain_idxs = idxs[m:]  # Indices for the retain set
forget_idxs = idxs[:m]  # Indices for the forget set

# Split the data based on indices
retain_set = Subset(trainset, retain_idxs)
forget_set = Subset(trainset, forget_idxs)

# Create DataLoader instances
batch_size = 64
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)
retain_loader = DataLoader(retain_set, batch_size=batch_size, shuffle=True)
forget_loader = DataLoader(forget_set, batch_size=batch_size, shuffle=False)

counter = Counter(y_train)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%s, Count=%d, Percentage=%.2f%%' % (k, v, per))
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
initial_model = densenet(num_classes=num_classes,depth=100,growthRate=12,compressionRate=2,dropRate=0)
criterion = nn.CrossEntropyLoss()
lr = 0.1
n_repeat = 1
max_epochs = 100
patience = None

Files already downloaded and verified
Files already downloaded and verified
Class=6, Count=5000, Percentage=10.00%
Class=9, Count=5000, Percentage=10.00%
Class=4, Count=5000, Percentage=10.00%
Class=1, Count=5000, Percentage=10.00%
Class=2, Count=5000, Percentage=10.00%
Class=7, Count=5000, Percentage=10.00%
Class=8, Count=5000, Percentage=10.00%
Class=3, Count=5000, Percentage=10.00%
Class=5, Count=5000, Percentage=10.00%
Class=0, Count=5000, Percentage=10.00%


# Differential privacy

In [6]:
# Step 1: k-anonymize and prepare D_k
ft_epochs_list = [10, 15, 20]
EPS = [0.5]
for epsilon in EPS:
    # Directory path for the corresponding epsilon dataset
    dataset_dir = f'dp_data/cifar10/m16_b4/dp_cifar10_eps_{epsilon}'

    # Use the original CIFAR-10 labels for DP-obfuscated images
    labels = trainset.targets
    # Create a dataset object for the DP-obfuscated dataset
    train_dataset_k = DPCIFAR10Dataset(root_dir=dataset_dir, labels=labels, transform=transform_train)
    # Create TensorDatasets
    train_loader_k = DataLoader(train_dataset_k, batch_size=batch_size, shuffle=True)

    train_accs_k = []
    test_accs_k = []
    mia_aucs_k = []
    mia_advs_k = []
    runtimes_train_k = []

    torch.cuda.empty_cache()
    model_k = copy.deepcopy(initial_model)
    optimizer = optim.SGD(model_k.parameters(), lr=lr, momentum=0.9)
    t0 = time.time()
    model_k = train_model(model_k, train_loader_k, test_loader, criterion, optimizer, 
                    max_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                    patience = patience)

    t1 = time.time()
    rt_train = t1- t0
    runtimes_train_k.append(rt_train)

    # Evaluate the model accuracy, and MIA
    model_k.eval()
    #Accuracy
    train_acc = accuracy(model_k, train_loader)
    test_acc = accuracy(model_k, test_loader)
    train_accs_k.append(100.0*train_acc)
    test_accs_k.append(100.0*test_acc)
    #MIA
    idxs = np.arange(len(testset))
    random.shuffle(idxs)
    rand_idxs = idxs[:m]
    logits_test, loss_test, test_labels = compute_attack_components(model_k, test_loader)
    logits_forget, loss_forget, forget_labels = compute_attack_components(model_k, forget_loader)
    attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                          forget_labels, test_labels[rand_idxs])
    auc = attack_result.get_result_with_max_auc().get_auc()
    adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
    mia_aucs_k.append(100.0*auc)
    mia_advs_k.append(100.0*adv)
    
    # Further finetune on D and Dr
    for ft_epochs in ft_epochs_list:
        train_accs_k_D = []
        test_accs_k_D = []
        mia_aucs_k_D = []
        mia_advs_k_D = []
        runtimes_train_k_D = []

        retain_accs_k_ret = []
        forget_accs_k_ret = []
        test_accs_k_ret = []
        mia_aucs_k_ret = []
        mia_advs_k_ret = []
        runtimes_train_k_ret = []
        
        model_k_D = copy.deepcopy(model_k)
        torch.cuda.empty_cache()
        optimizer = optim.Adam(model_k_D.parameters(), lr=lr/10)
        t0 = time.time()
        model_k_D = train_model(model_k_D, train_loader, test_loader, criterion, optimizer, 
                            ft_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                              patience = patience)

        t1 = time.time()
        rt = t1-t0
        runtimes_train_k_D.append(rt)

        # Evaluate the model accuracy, and MIA
        model_k_D.eval()
        #Accuracy
        train_acc = accuracy(model_k_D, train_loader)
        test_acc = accuracy(model_k_D, test_loader)
        train_accs_k_D.append(100.0*train_acc)
        test_accs_k_D.append(100.0*test_acc)
        #MIA
        logits_test, loss_test, test_labels = compute_attack_components(model_k_D, test_loader)
        logits_forget, loss_forget, forget_labels = compute_attack_components(model_k_D, forget_loader)
        attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                              forget_labels, test_labels[rand_idxs])
        auc = attack_result.get_result_with_max_auc().get_auc()
        adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
        mia_aucs_k_D.append(100.0*auc)
        mia_advs_k_D.append(100.0*adv)

        model_k_ret = copy.deepcopy(model_k)
        torch.cuda.empty_cache()
        optimizer = optim.Adam(model_k_ret.parameters(), lr=lr/10)
        t0 = time.time()
        model_k_ret = train_model(model_k_ret, retain_loader, test_loader, criterion, optimizer, 
                            ft_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                              patience = patience)

        t1 = time.time()
        rt = t1-t0
        runtimes_train_k_ret.append(rt)
        # Evaluate the model accuracy, and MIA
        model_k_ret.eval()
        #Accuracy
        retain_acc = accuracy(model_k_ret, retain_loader)
        forget_acc = accuracy(model_k_ret, forget_loader)
        test_acc = accuracy(model_k_ret, test_loader)
        retain_accs_k_ret.append(100.0*retain_acc)
        forget_accs_k_ret.append(100.0*forget_acc)
        test_accs_k_ret.append(100.0*test_acc)
        #MIA
        logits_test, loss_test, test_labels = compute_attack_components(model_k_ret, test_loader)
        logits_forget, loss_forget, forget_labels = compute_attack_components(model_k_ret, forget_loader)
        attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                              forget_labels, test_labels[rand_idxs])
        auc = attack_result.get_result_with_max_auc().get_auc()
        adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
        mia_aucs_k_ret.append(100.0*auc)
        mia_advs_k_ret.append(100.0*adv)

        # Anonymizing D and training M_k on D_k
        mean_train_k_time = np.mean(runtimes_train_k)
        std_train_k_time = np.std(runtimes_train_k)
        mean_train_k_acc = np.mean(train_accs_k)
        std_train_k_acc = np.std(train_accs_k)
        mean_test_k_acc = np.mean(test_accs_k)
        std_test_k_acc = np.std(test_accs_k)
        mean_mia_k_auc = np.mean(mia_aucs_k)
        std_mia_k_auc = np.std(mia_aucs_k)
        mean_mia_k_adv = np.mean(mia_advs_k)
        std_mia_k_adv = np.std(mia_advs_k)

        # Finetuning M_k on D
        mean_finetune_D_time = np.mean(runtimes_train_k_D)
        std_finetune_D_time = np.std(runtimes_train_k_D)
        mean_finetune_D_train_acc = np.mean(train_accs_k_D)
        std_finetune_D_train_acc = np.std(train_accs_k_D)
        mean_finetune_D_test_acc = np.mean(test_accs_k_D)
        std_finetune_D_test_acc = np.std(test_accs_k_D)
        mean_finetune_D_mia_auc = np.mean(mia_aucs_k_D)
        std_finetune_D_mia_auc = np.std(mia_aucs_k_D)
        mean_finetune_D_mia_adv = np.mean(mia_advs_k_D)
        std_finetune_D_mia_adv = np.std(mia_advs_k_D)

        # Finetuning M_k on D_ret
        mean_finetune_D_ret_time = np.mean(runtimes_train_k_ret)
        std_finetune_D_ret_time = np.std(runtimes_train_k_ret)
        mean_finetune_D_ret_train_acc = np.mean(retain_accs_k_ret)
        std_finetune_D_ret_train_acc = np.std(retain_accs_k_ret)
        mean_finetune_D_ret_forget_acc = np.mean(forget_accs_k_ret)
        std_finetune_D_ret_forget_acc = np.std(forget_accs_k_ret)
        mean_finetune_D_ret_test_acc = np.mean(test_accs_k_ret)
        std_finetune_D_ret_test_acc = np.std(test_accs_k_ret)
        mean_finetune_D_ret_mia_auc = np.mean(mia_aucs_k_ret)
        std_finetune_D_ret_mia_auc = np.std(mia_aucs_k_ret)
        mean_finetune_D_ret_mia_adv = np.mean(mia_advs_k_ret)
        std_finetune_D_ret_mia_adv = np.std(mia_advs_k_ret)

        # Print the results
        print('----------------------------------------')
        print('Epsilon=', epsilon, 'Fine-tuning epochs=', ft_epochs)
        print('----------------------------------------')
        print('-----Anonymizing D and training M_dp on D_dp-----')
        print('Training M_k on D_k time:{:0.2f}(±{:0.2f})'.format(mean_train_k_time, std_train_k_time))
        print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_train_k_acc, std_train_k_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_test_k_acc, std_test_k_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_auc, std_mia_k_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_adv, std_mia_k_adv))

        print('-----Finetuning M_k on D-----')
        print('Training M_k on D time:{:0.2f}(±{:0.2f})'.format(mean_finetune_D_time, std_finetune_D_time))
        print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_train_acc, std_finetune_D_train_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_test_acc, std_finetune_D_test_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_auc, std_finetune_D_mia_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_adv, std_finetune_D_mia_adv))

        print('-----Finetuning M_k on D_ret-----')
        print('Finetuning M_k on D_retain time:{:0.2f}(±{:0.2f}) seconds'.format(mean_finetune_D_ret_time, std_finetune_D_ret_time))
        print('Retain accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc))
        print('Forget accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv))
        print('----------------------------------------')

#         # Save to CSV
#         csv_anonymize_file_path = 'results/cifar10/densenet12_mdp_eps={}_fr={}.csv'.format(epsilon, forget_ratio)
#         csv_finetune_D_file_path = 'results/cifar10/densenet12_mdpd_eps={}_fr={}_epochs={}.csv'.format(epsilon, forget_ratio, ft_epochs)
#         csv_finetune_D_ret_file_path = 'results/cifar10/densenet12_mdpret_eps={}_fr={}_epochs={}.csv'.format(epsilon, forget_ratio, ft_epochs)

#         # Writing to CSV for anonymizing, finetuning on D, and finetuning on D_ret
#         with open(csv_anonymize_file_path, mode='w', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
#             writer.writerow(['Training M_k on D_k Time', mean_train_k_time, std_train_k_time])
#             writer.writerow(['Train Accuracy', mean_train_k_acc, std_train_k_acc])
#             writer.writerow(['Test Accuracy', mean_test_k_acc, std_test_k_acc])
#             writer.writerow(['MIA AUC', mean_mia_k_auc, std_mia_k_auc])
#             writer.writerow(['MIA Advantage', mean_mia_k_adv, std_mia_k_adv])

#         with open(csv_finetune_D_file_path, mode='w', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
#             writer.writerow(['Training M_k on D Time', mean_finetune_D_time, std_finetune_D_time])
#             writer.writerow(['Train Accuracy', mean_finetune_D_train_acc, std_finetune_D_train_acc])
#             writer.writerow(['Test Accuracy', mean_finetune_D_test_acc, std_finetune_D_test_acc])
#             writer.writerow(['MIA AUC', mean_finetune_D_mia_auc, std_finetune_D_mia_auc])
#             writer.writerow(['MIA Advantage', mean_finetune_D_mia_adv, std_finetune_D_mia_adv])

#         with open(csv_finetune_D_ret_file_path, mode='w', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
#             writer.writerow(['Finetuning M_k on D_retain Time', mean_finetune_D_ret_time, std_finetune_D_ret_time])
#             writer.writerow(['Retain Accuracy', mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc])
#             writer.writerow(['Forget Accuracy', mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc])
#             writer.writerow(['Test Accuracy', mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc])
#             writer.writerow(['MIA AUC', mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc])
#             writer.writerow(['MIA Advantage', mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv])

100%|████████████████████████████████████████████████████████████████████████████████| 782/782 [04:45<00:00,  2.74it/s]


Epoch [1/100] | Train Loss: 2.316596520831213


100%|████████████████████████████████████████████████████████████████████████████████| 782/782 [04:43<00:00,  2.76it/s]


Epoch [10/100] | Train Loss: 2.211079520337722


100%|████████████████████████████████████████████████████████████████████████████████| 782/782 [04:44<00:00,  2.75it/s]


...

Epoch [1/20] | Train Loss: 1.7867550566732322


100%|████████████████████████████████████████████████████████████████████████████████| 743/743 [05:16<00:00,  2.34it/s]


Epoch [10/20] | Train Loss: 0.16366729268385535


100%|████████████████████████████████████████████████████████████████████████████████| 743/743 [05:16<00:00,  2.35it/s]


Epoch [20/20] | Train Loss: 0.0010829838564777514
----------------------------------------
Epsilon= 0.5 Fine-tuning epochs= 20
----------------------------------------
-----Anonymizing D and training M_dp on D_dp-----
Training M_k on D_k time:28451.27(±0.00)
Train accuracy:13.89(±0.00)%
Test accuracy:14.17(±0.00)%
MIA AUC:50.54(±0.00)%
MIA Advantage:2.72(±0.00)%
-----Finetuning M_k on D-----
Training M_k on D time:6641.32(±0.00)
Train accuracy:100.00(±0.00)%
Test accuracy:88.89(±0.00)%
MIA AUC:67.04(±0.00)%
MIA Advantage:27.32(±0.00)%
-----Finetuning M_k on D_ret-----
Finetuning M_k on D_retain time:6294.21(±0.00) seconds
Retain accuracy:100.00(±0.00)%
Forget accuracy:89.20(±0.00)%
Test accuracy:88.03(±0.00)%
MIA AUC:51.07(±0.00)%
MIA Advantage:2.56(±0.00)%
----------------------------------------
